# WRC Live Timing API Scraper

Simple exploration of grabbing data from WRC Live Timing API.

In [209]:
from urllib.parse import urljoin
from parse import parse

import pandas as pd
import requests

#%pip install requests-cache
import requests_cache
requests_cache.install_cache("demo_cache")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 2.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [116]:
WRC_API_BASE = "https://api.wrc.com/content/{path}"

CURRENT_YEAR = 2023
CURRENT_SEASON = 20

seasonId = CURRENT_SEASON

In [229]:
from itertools import zip_longest


def _WRC_json(path, base=WRC_API_BASE):
    """Return JSON from API."""
    url = urljoin(base, path)
    #print(f"Fetching: {url}")
    r = requests.get(url)
    rj = r.json()
    if "status" in rj and rj["status"]=="Not Found":
        return {}
    return r.json()


def convert_date_range(date_range_str):
    """Convert date of from `19 - 22 JAN 2023` to date range."""
    r = parse("{start_day} - {end_day} {month} {year}", date_range_str)
    start_date = pd.to_datetime(
        f"{r['start_day']} {r['month']} {r['year']}", format="%d %b %Y"
    )
    end_date = pd.to_datetime(
        f"{r['end_day']} {r['month']} {r['year']}", format="%d %b %Y"
    )
    return pd.date_range(start=start_date, end=end_date)


def timeify(df, col, typ=None):
    """Convert a column  to a datetime inplace."""
    if typ == "daterange":
        df[col] = df[col].apply(convert_date_range)
    else:
        df[col] = pd.to_datetime(df[col].astype(int), unit="ms")


def tablify(json_data, subcolkey=None):
    """Generate table from separate colnames/values JSON."""
    # Note that the JSON may be a few rows short cf. provided keys
    if subcolkey is None:
        results = []
        for values in json_data["values"]:
            result_dict = {}
            # Zip keys and values, filling None for missing values
            zipped_values = zip_longest(json_data["fields"], values, fillvalue=None)
            # Convert the zipped values to a dictionary and update the result_dict
            result_dict.update(dict(zipped_values))
            results.append(result_dict)
        return pd.DataFrame(results)
    else:
        df = pd.DataFrame(columns=json_data["fields"])
        for value in json_data["values"]:
            _df = pd.DataFrame(value[subcolkey])
            if len(_df.columns) < len(json_data["fields"]):
                _df[[json_data["fields"][len(_df.columns) :]]] = None
            _df.columns = json_data["fields"]
            for c in [k for k in value.keys() if k != subcolkey]:
                _df[c] = value[c]
                df = pd.concat([df, _df])
        return df

## Season Calendar

The API call `https://api.wrc.com/content/filters/calendar` gives a list of current or latest (last, if season's end) events, from which we can get a season identifer. For the 2023 WRC championship, the season ID is `20`

In [230]:
def getFullCalendar(year=CURRENT_YEAR):
    stub = (
        f"filters/calendar?language=en&size=20&championship=wrc&origin=vcms&year={year}"
    )
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.dataFrame()
    # return json_data
    return pd.DataFrame(json_data["content"])


df_fullcal = getFullCalendar()
df_fullcal.head()

Fetching: https://api.wrc.com/content/filters/calendar?language=en&size=20&championship=wrc&origin=vcms&year=2023


,id,guid,title,location,startDate,endDate,eventId,rallyId,description,round,...,uid,seriesUid,releaseYear,availableOn,availableTill,startDateLocal,endDateLocal,finishDate,championship,championshipLogo
0,IQXg,WRC_2023_01,Rallye Monte-Carlo,Monaco,1674125280000,1674384480000,353,357,The most unpredictable rally of the year. Rela...,1,...,IQXg,WRC_2023_01,2023,1674125280000,1674384480000,2023-01-19T11:48:00+01:00,2023-01-22T11:48:00+01:00,1674384480000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
1,lGzi,WRC_2023_02,WRC Rally Sweden,Umea,1675940100000,1676199300000,354,358,The WRC’s ultimate winter challenge. Watch in ...,2,...,lGzi,WRC_2023_02,2023,1675940100000,1676199300000,2023-02-09T11:55:00+01:00,2023-02-12T11:55:00+01:00,1676199300000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
2,siDN,WRC_2023_03,WRC Guanajuato Rally México,"Leon, Guanajuato",1678949100000,1679208300000,355,359,A thrilling ride through the colourful landsca...,3,...,siDN,WRC_2023_03,2023,1678949100000,1679208300000,2023-03-16T00:45:00-06:00,2023-03-19T00:45:00-06:00,1679208300000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
3,rejA,WRC_2023_04,WRC Croatia Rally,"Zagreb, City of Zagreb Region",1682059680000,1682232480000,356,360,High-speed stretches meet challenging hairpin ...,4,...,rejA,WRC_2023_04,2023,1682059680000,1682232480000,2023-04-21T08:48:00+02:00,2023-04-23T08:48:00+02:00,1682232480000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
4,9xIl,WRC_2023_05,WRC Vodafone Rally de Portugal,"Matosinhos, Porto",1683870600000,1684043400000,357,361,Fast but technical gravel roads inland from Po...,5,...,9xIl,WRC_2023_05,2023,1683870600000,1684043400000,2023-05-12T06:50:00+01:00,2023-05-14T06:50:00+01:00,1684043400000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...


In [231]:
df_fullcal

,id,guid,title,location,startDate,endDate,eventId,rallyId,description,round,...,uid,seriesUid,releaseYear,availableOn,availableTill,startDateLocal,endDateLocal,finishDate,championship,championshipLogo
0,IQXg,WRC_2023_01,Rallye Monte-Carlo,Monaco,1674125280000,1674384480000,353,357,The most unpredictable rally of the year. Rela...,1,...,IQXg,WRC_2023_01,2023,1674125280000,1674384480000,2023-01-19T11:48:00+01:00,2023-01-22T11:48:00+01:00,1674384480000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
1,lGzi,WRC_2023_02,WRC Rally Sweden,Umea,1675940100000,1676199300000,354,358,The WRC’s ultimate winter challenge. Watch in ...,2,...,lGzi,WRC_2023_02,2023,1675940100000,1676199300000,2023-02-09T11:55:00+01:00,2023-02-12T11:55:00+01:00,1676199300000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
2,siDN,WRC_2023_03,WRC Guanajuato Rally México,"Leon, Guanajuato",1678949100000,1679208300000,355,359,A thrilling ride through the colourful landsca...,3,...,siDN,WRC_2023_03,2023,1678949100000,1679208300000,2023-03-16T00:45:00-06:00,2023-03-19T00:45:00-06:00,1679208300000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
3,rejA,WRC_2023_04,WRC Croatia Rally,"Zagreb, City of Zagreb Region",1682059680000,1682232480000,356,360,High-speed stretches meet challenging hairpin ...,4,...,rejA,WRC_2023_04,2023,1682059680000,1682232480000,2023-04-21T08:48:00+02:00,2023-04-23T08:48:00+02:00,1682232480000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
4,9xIl,WRC_2023_05,WRC Vodafone Rally de Portugal,"Matosinhos, Porto",1683870600000,1684043400000,357,361,Fast but technical gravel roads inland from Po...,5,...,9xIl,WRC_2023_05,2023,1683870600000,1684043400000,2023-05-12T06:50:00+01:00,2023-05-14T06:50:00+01:00,1684043400000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
5,BTVM,WRC_2023_06,WRC Rally Italia Sardegna,"Olbia, Sardinia",1685602260000,1685861460000,358,362,Don’t let the picturesque Mediterranean backdr...,6,...,BTVM,WRC_2023_06,2023,1685602260000,1685861460000,2023-06-01T08:51:00+02:00,2023-06-04T08:51:00+02:00,1685861460000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
6,vhG4,WRC_2023_07,WRC Safari Rally Kenya,"Naivasha, Kenya",1687416720000,1687675920000,359,363,Held on the untamed terrains of Africa’s breat...,7,...,vhG4,WRC_2023_07,2023,1687416720000,1687675920000,2023-06-22T09:52:00+03:00,2023-06-25T09:52:00+03:00,1687675920000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
7,38Ly,WRC_2023_08,WRC Rally Estonia,Tartu,1689836040000,1690095240000,360,364,Mind-bendingly fast smooth gravel stages inclu...,8,...,38Ly,WRC_2023_08,2023,1689836040000,1690095240000,2023-07-20T09:54:00+03:00,2023-07-23T09:54:00+03:00,1690095240000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
8,Kf3P,WRC_2023_09,WRC Secto Automotive Rally Finland,Jyväskylä,1691001000000,1691346540000,361,365,A mecca for rally drivers and enthusiasts alik...,9,...,Kf3P,WRC_2023_09,2023,1691001000000,1691346540000,2023-08-02T21:30:00+03:00,2023-08-06T21:29:00+03:00,1691346540000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
9,aCfi,WRC_2023_10,WRC EKO Acropolis Rally Greece,Lamia,1694097840000,1694346300000,362,366,A legendary WRC fixture. Twisty gravel mountai...,10,...,aCfi,WRC_2023_10,2023,1694097840000,1694346300000,2023-09-07T17:44:00+03:00,2023-09-10T14:45:00+03:00,1694346300000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...


In [232]:
# https://api.wrc.com/content/result/calendar?season=20&championship=wrc


def getResultsCalendar(seasonId=CURRENT_SEASON):
    """Get the WRC Calendar for a given season ID as a JSON result."""
    stub = f"result/calendar?season={seasonId}&championship=wrc"
    json_data = _WRC_json(stub)
    df_calendar = tablify(json_data)
    #timeify(df_calendar, "date", "daterange")
    #timeify(df_calendar, "startDate")
    #timeify(df_calendar, "finishDate")
    # df_calendar.set_index("id", inplace=True)
    return df_calendar

In [233]:
getResultsCalendar().head()

Fetching: https://api.wrc.com/content/result/calendar?season=20&championship=wrc


,id,rallyTitle,ROUND,rallyCountry,rallyCountryImage,rallyId,date,startDate,finishDate,driverId,driverCountryImage,driver,coDriverId,coDriverCountryImage,coDriver,teamId,teamLogo,teamName,manufacturer
0,IQXg,Rallye Monte-Carlo,1,Monaco,Flags/MCO.png,357,19 - 22 JAN 2023,1674125280000,1674384480000,1cf9ade6-25b5-5586-8393-eacfeb943eae,Flags/FRA.png,Sébastien OGIER,867cd58f-3fe3-5290-9cc5-b21cb41c523c,Flags/FRA.png,Vincent LANDAIS,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,Flags/toyota.png,TOYOTA GAZOO RACING WRT,Toyota
1,lGzi,WRC Rally Sweden,2,Sweden,Flags/SWE.png,358,09 - 12 FEB 2023,1675940100000,1676199300000,6632e7ca-34bf-55b8-9cad-d060000fa794,Flags/EST.png,Ott TÄNAK,00a8a5c3-f7ba-5086-86df-1a59b7da7e26,Flags/EST.png,Martin JÄRVEOJA,887309d0-48be-5f83-ad3e-abf2a79a64a3,Flags/ford.png,M-SPORT FORD WORLD RALLY TEAM,Ford
2,siDN,WRC Guanajuato Rally México,3,Mexico,Flags/MEX.png,359,16 - 19 MAR 2023,1678949100000,1679208300000,1cf9ade6-25b5-5586-8393-eacfeb943eae,Flags/FRA.png,Sébastien OGIER,867cd58f-3fe3-5290-9cc5-b21cb41c523c,Flags/FRA.png,Vincent LANDAIS,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,Flags/toyota.png,TOYOTA GAZOO RACING WRT,Toyota
3,rejA,WRC Croatia Rally,4,Croatia,Flags/HRV.png,360,21 - 23 APR 2023,1682059680000,1682232480000,ae7329c9-79b3-5d96-886c-22cca6217764,Flags/GBR.png,Elfyn EVANS,53e56691-fe7c-5271-9dc5-8960df28b221,Flags/GBR.png,Scott MARTIN,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,Flags/toyota.png,TOYOTA GAZOO RACING WRT,Toyota
4,9xIl,WRC Vodafone Rally de Portugal,5,Portugal,Flags/PRT.png,361,12 - 14 MAY 2023,1683870600000,1684043400000,d8e4bbea-3af2-5486-9ad5-a445aaec573e,Flags/FIN.png,Kalle ROVANPERÄ,e4dd8a3f-00e9-59f7-9871-9337af6085d7,Flags/FIN.png,Jonne HALTTUNEN,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,Flags/toyota.png,TOYOTA GAZOO RACING WRT,Toyota


In [234]:
eventId, rallyId = df_fullcal.iloc[-1][["eventId", "rallyId"]]
eventId, rallyId

('365', '369')

In [235]:
## Stages


def getStageDetails(eventId, rallyId):
    stub = f"result/stages?eventId={eventId}&rallyId={rallyId}&championship=wrc"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.dataFrame()
    df_stageDetails = tablify(json_data)
    return df_stageDetails


df_stageDetails = getStageDetails(eventId, rallyId)
df_stageDetails.head(3)

Fetching: https://api.wrc.com/content/result/stages?eventId=365&rallyId=369&championship=wrc


,id,STAGE,STAGE TYPE,stageId,eventId,STATUS,day,name,distance
0,ee55daa1-fd07-574f-835a-f39008081b95,SHD,shakedown,SHD,365,Complete,,Shakedown,
1,3cd7a38a-436f-5c21-b966-8b4743f20a57,SS1,HeadToHeadSuperSpecialStage,6119,365,Completed,Thursday,SS1 TOYOTA STADIUM SSS 1 (2.1km),2.1
2,7ae2243b-6746-516b-895e-16b5d0a2ed51,SS2,SpecialStage,6120,365,Interrupted,Friday,SS2 Isegami's Tunnel 1 (23.67km),23.67


In [252]:
def getOverall(eventId, rallyId, stageId, championship="wrc"):
    stub = f"result/stageResult?eventId={eventId}&rallyId={rallyId}&stageId={stageId}&championship={championship}"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_overall = tablify(json_data)
    return df_overall


stageId = df_stageDetails.iloc[-1]["stageId"]

getOverall(eventId, rallyId, stageId).head(3)

Fetching: https://api.wrc.com/content/result/stageResult?eventId=365&rallyId=369&stageId=FINAL&championship=wrc


,id,pos,carNo,driverId,driverCountry,driverCountryImage,driver,coDriverId,coDriverCountry,coDriverCountryImage,...,team/car,teamName,teamLogo,eligibility,stageTime,penaltyTime,totalTime,diffFirst,diffPrev,groupClass
0,c991bcbd-6d9a-5efc-8bde-53d1c3bacae5,1,#33,ae7329c9-79b3-5d96-886c-22cca6217764,United Kingdom of Great Britain and Northern I...,Flags/GBR.png,Elfyn EVANS,53e56691-fe7c-5271-9dc5-8960df28b221,United Kingdom of Great Britain and Northern I...,Flags/GBR.png,...,GR Yaris Rally1 HYBRID,Toyota,teamLogo/toyota.png,M,3:32:08.8,,3:32:08.8,,,RC1
1,2141d91f-8620-5848-8873-32f48a37f2d7,2,#17,1cf9ade6-25b5-5586-8393-eacfeb943eae,France,Flags/FRA.png,Sébastien OGIER,867cd58f-3fe3-5290-9cc5-b21cb41c523c,France,Flags/FRA.png,...,GR Yaris Rally1 HYBRID,Toyota,teamLogo/toyota.png,M,3:32:26.5,1:00.0,3:33:26.5,+1:17.7,+1:17.7,RC1
2,0b585f28-25f4-5d7f-be34-6ef26f0e086e,3,#69,d8e4bbea-3af2-5486-9ad5-a445aaec573e,Finland,Flags/FIN.png,Kalle ROVANPERÄ,e4dd8a3f-00e9-59f7-9871-9337af6085d7,Finland,Flags/FIN.png,...,GR Yaris Rally1 HYBRID,Toyota,teamLogo/toyota.png,M,3:33:55.3,,3:33:55.3,+1:46.5,+28.8,RC1


In [237]:
def getStageTimes(eventId, rallyId, stageId, championship="wrc"):
    stub = f"result/stageTimes?eventId={eventId}&rallyId={rallyId}&stageId={stageId}&championship={championship}"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_stageTimes = tablify(json_data)
    return df_stageTimes


getStageTimes(eventId, rallyId, stageId).head(3)

Fetching: https://api.wrc.com/content/result/stageTimes?eventId=365&rallyId=369&stageId=FINAL&championship=wrc


,id,pos,carNo,driverId,driverCountry,driverCountryImage,driver,coDriverId,coDriverCountry,coDriverCountryImage,...,team/car,teamName,teamLogo,eligibility,stageTime,penaltyTime,totalTime,diffFirst,diffPrev,groupClass
0,c991bcbd-6d9a-5efc-8bde-53d1c3bacae5,1,#33,ae7329c9-79b3-5d96-886c-22cca6217764,United Kingdom of Great Britain and Northern I...,Flags/GBR.png,Elfyn EVANS,53e56691-fe7c-5271-9dc5-8960df28b221,United Kingdom of Great Britain and Northern I...,Flags/GBR.png,...,GR Yaris Rally1 HYBRID,Toyota,teamLogo/toyota.png,M,3:32:08.8,,3:32:08.8,,,RC1
1,2141d91f-8620-5848-8873-32f48a37f2d7,2,#17,1cf9ade6-25b5-5586-8393-eacfeb943eae,France,Flags/FRA.png,Sébastien OGIER,867cd58f-3fe3-5290-9cc5-b21cb41c523c,France,Flags/FRA.png,...,GR Yaris Rally1 HYBRID,Toyota,teamLogo/toyota.png,M,3:32:26.5,1:00.0,3:33:26.5,+1:17.7,+1:17.7,RC1
2,0b585f28-25f4-5d7f-be34-6ef26f0e086e,3,#69,d8e4bbea-3af2-5486-9ad5-a445aaec573e,Finland,Flags/FIN.png,Kalle ROVANPERÄ,e4dd8a3f-00e9-59f7-9871-9337af6085d7,Finland,Flags/FIN.png,...,GR Yaris Rally1 HYBRID,Toyota,teamLogo/toyota.png,M,3:33:55.3,,3:33:55.3,+1:46.5,+28.8,RC1


In [238]:
getStageTimes(365, 369, 4161).head(3)

Fetching: https://api.wrc.com/content/result/stageTimes?eventId=365&rallyId=369&stageId=4161&championship=wrc


""


In [239]:
def getSplitTimes(eventId, rallyId, stageId, championship="wrc"):
    stub = f"result/splitTime?eventId={eventId}&rallyId={rallyId}&stageId={stageId}&championship={championship}"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_splitTimes = tablify(json_data)
    return df_splitTimes


getSplitTimes(eventId, rallyId, stageId).head(3)

Fetching: https://api.wrc.com/content/result/splitTime?eventId=365&rallyId=369&stageId=FINAL&championship=wrc


,id,pos,carNo,driverId,driverCountry,driverCountryImage,driver,coDriverId,coDriverCountry,coDriverCountryImage,...,team/car,teamName,teamLogo,eligibility,stageTime,penaltyTime,totalTime,diffFirst,diffPrev,groupClass
0,c991bcbd-6d9a-5efc-8bde-53d1c3bacae5,1,#33,ae7329c9-79b3-5d96-886c-22cca6217764,United Kingdom of Great Britain and Northern I...,Flags/GBR.png,Elfyn EVANS,53e56691-fe7c-5271-9dc5-8960df28b221,United Kingdom of Great Britain and Northern I...,Flags/GBR.png,...,GR Yaris Rally1 HYBRID,Toyota,teamLogo/toyota.png,M,3:32:08.8,,3:32:08.8,,,RC1
1,2141d91f-8620-5848-8873-32f48a37f2d7,2,#17,1cf9ade6-25b5-5586-8393-eacfeb943eae,France,Flags/FRA.png,Sébastien OGIER,867cd58f-3fe3-5290-9cc5-b21cb41c523c,France,Flags/FRA.png,...,GR Yaris Rally1 HYBRID,Toyota,teamLogo/toyota.png,M,3:32:26.5,1:00.0,3:33:26.5,+1:17.7,+1:17.7,RC1
2,0b585f28-25f4-5d7f-be34-6ef26f0e086e,3,#69,d8e4bbea-3af2-5486-9ad5-a445aaec573e,Finland,Flags/FIN.png,Kalle ROVANPERÄ,e4dd8a3f-00e9-59f7-9871-9337af6085d7,Finland,Flags/FIN.png,...,GR Yaris Rally1 HYBRID,Toyota,teamLogo/toyota.png,M,3:33:55.3,,3:33:55.3,+1:46.5,+28.8,RC1


In [240]:
getSplitTimes(eventId, rallyId, stageId).columns

Fetching: https://api.wrc.com/content/result/splitTime?eventId=365&rallyId=369&stageId=FINAL&championship=wrc


Index(['id', 'pos', 'carNo', 'driverId', 'driverCountry', 'driverCountryImage',
       'driver', 'coDriverId', 'coDriverCountry', 'coDriverCountryImage',
       'coDriver', 'teamId', 'team/car', 'teamName', 'teamLogo', 'eligibility',
       'stageTime', 'penaltyTime', 'totalTime', 'diffFirst', 'diffPrev',
       'groupClass'],
      dtype='object')

In [241]:
def getStageWinners(eventId):
    stub = f"result/stageWinners?eventId={eventId}"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_stageWinners = tablify(json_data)
    return df_stageWinners


getStageWinners(eventId).head(3)

Fetching: https://api.wrc.com/content/result/stageWinners?eventId=365


,id,carNo,stageNo,stageName,stageType,stageId,eventId,entryId,driverId,driverCountry,...,coDriverId,coDriverCountry,coDriverCountryImage,coDriver,teamId,team/car,teamName,teamLogo,eligibility,time
0,365,#11,SS1,TOYOTA STADIUM SSS 1 (2.1 km),HeadToHeadSuperSpecialStage,6119,a0d04378-f379-53ff-a3b2-aa8b3e27a3f0,6004e998-29a1-5a2e-b020-548227878bfe,c99a2a26-bd03-5153-aaa7-684d3acb5491,Belgium,...,b1b98699-0332-528a-8a80-11eb538f1ded,Belgium,Flags/BEL.png,Martijn WYDAEGHE,b6692ea5-df92-5cad-a91c-20319a6fffd7,i20 N Rally1 HYBRID,Hyundai,teamLogo/hyundai.png,M,1:47.6
1,365,#33,SS2,Isegami's Tunnel 1 (23.67 km),SpecialStage,6120,a0d04378-f379-53ff-a3b2-aa8b3e27a3f0,94ad0762-7c1b-53b9-85e3-c39e88f4d105,ae7329c9-79b3-5d96-886c-22cca6217764,United Kingdom of Great Britain and Northern I...,...,53e56691-fe7c-5271-9dc5-8960df28b221,United Kingdom of Great Britain and Northern I...,Flags/GBR.png,Scott MARTIN,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,GR Yaris Rally1 HYBRID,Toyota,teamLogo/toyota.png,M,20:17.8
2,365,#33,SS3,Inabu Dam 1 (19.38 km),SpecialStage,6121,a0d04378-f379-53ff-a3b2-aa8b3e27a3f0,94ad0762-7c1b-53b9-85e3-c39e88f4d105,ae7329c9-79b3-5d96-886c-22cca6217764,United Kingdom of Great Britain and Northern I...,...,53e56691-fe7c-5271-9dc5-8960df28b221,United Kingdom of Great Britain and Northern I...,Flags/GBR.png,Scott MARTIN,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,GR Yaris Rally1 HYBRID,Toyota,teamLogo/toyota.png,M,14:20.0


In [242]:
def getItinerary(eventId):
    stub = f"result/itinerary?eventId={eventId}&extended=false"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_itinerary = tablify(json_data, "values")
    return df_itinerary


getItinerary(eventId).tail(3)

Fetching: https://api.wrc.com/content/result/itinerary?eventId=365&extended=false
namecols 4


,stage,eventId,stageId,distance,timingPrecision,firstCarDueDateTime,firstCarDueDateTimeMs,controlPenalties,status,type,location,targetDuration,targetDurationMs,id,order,date
3,SS20,365,128c0d05-4538-536f-a7d2-f929836d1dc4,22.92 km,Minute,11:04,2023-11-19T11:04:00+09:00,None,Completed,StageStart,Ena City 2,00:03:00,180000,1095.0,4.0,Sunday 19th November
4,SS21,365,a9e4c371-57ba-5c72-8fff-8b92a95baa60,11.6 km,Minute,11:57,2023-11-19T11:57:00+09:00,None,Completed,StageStart,Nenoue Kougen 2,00:03:00,180000,1095.0,4.0,Sunday 19th November
5,SS22,365,4a0db031-0143-5756-af1f-e87945227001,7.52 km,Minute,14:15,2023-11-19T14:15:00+09:00,None,Completed,StageStart,Wolf Power Stage Asahi Kougen 2,00:03:00,180000,1095.0,4.0,Sunday 19th November


In [243]:
def getStartlist(eventId):
    stub = f"result/startLists?eventId={eventId}"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_startlist = tablify(json_data, "startListItems")
    return df_startlist


getStartlist(eventId).tail(3)

Fetching: https://api.wrc.com/content/result/startLists?eventId=365
namecols 5


,order,startDateTimeLocal,carNo,driverId,driverCountry,driverCountryImage,driver,coDriverId,coDriverCountry,coDriverCountryImage,coDriver,teamId,team/car,teamName,teamLogo,eligibility,groupClass,priority,id,date
29,30,2023-11-19T05:34:00+09:00,#45,3ab588df-cc06-5017-8ba6-143de81a5972,Japan,Flags/JPN.png,Koji TAKATA,a655075d-d4b8-5790-b96c-ed6480de882c,Japan,Flags/JPN.png,Takashi SEKIMOTO,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,GR Yaris,Toyota,teamLogo/toyota.png,,JRCar1,None,d5126c53-c54e-514f-8b16-16ff6b9b1050,Sunday 2023-11-19
30,31,2023-11-19T05:34:00+09:00,#47,8584c859-11be-5de8-9ba1-f91976ec0b46,Japan,Flags/JPN.png,Satoshi IRIE,7336f886-56da-5a98-b9a8-fc626172e583,Japan,Flags/JPN.png,Satoshi KAGEYAMA,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,Vitz,Toyota,teamLogo/toyota.png,,JRCar3,None,d5126c53-c54e-514f-8b16-16ff6b9b1050,Sunday 2023-11-19
31,32,2023-11-19T05:34:00+09:00,#48,4f12d0bb-2cbd-586a-86df-0844b79ef0d3,Japan,Flags/JPN.png,Hiroyasu SHIMIZU,bba42333-aeba-5102-8f37-191cf8de0723,Japan,Flags/JPN.png,Takahiro YASUI,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,Yaris,Toyota,teamLogo/toyota.png,,JRCar3,None,d5126c53-c54e-514f-8b16-16ff6b9b1050,Sunday 2023-11-19


In [244]:
def getPenalties(eventId):
    stub = f"result/penalty?eventId={eventId}"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_penalties = tablify(json_data)
    return df_penalties


getPenalties(eventId).head(3)

Fetching: https://api.wrc.com/content/result/penalty?eventId=365


,id,carNo,driverId,driverCountry,driverCountryImage,driver,coDriverId,coDriverCountry,coDriverCountryImage,coDriver,...,eligibility,groupClass,rallyId,entryId,controlId,penaltyTime,penaltyDuration,penaltyDurationMs,reason,control
0,4844,#46,e2464489-5926-5ac0-94a3-bdbf823bb827,Japan,Flags/JPN.png,Yasunori HAGIWARA,e6f37ce0-7731-5942-996d-63cde99f1e23,Japan,Flags/JPN.png,Koichi HARADA,...,,JRCar1,369,47504,25793,40.0,PT40S,40000,4 MINS LATE,TC1
1,4845,#34,d6030311-c622-5262-8a0e-8fb4e30bcc29,Japan,Flags/JPN.png,Toshi ARAI,05b11bf7-a75c-53e7-87eb-9ea080333b1c,Japan,Flags/JPN.png,Yuichi MATSUMOTO,...,,JRCar1,369,47494,25765,10.0,PT10S,10000,1 MIN LATE,TC1D
2,4846,#34,d6030311-c622-5262-8a0e-8fb4e30bcc29,Japan,Flags/JPN.png,Toshi ARAI,05b11bf7-a75c-53e7-87eb-9ea080333b1c,Japan,Flags/JPN.png,Yuichi MATSUMOTO,...,,JRCar1,369,47494,25765,10.0,PT10S,10000,1 MIN LATE,TC1D


In [245]:
def getRetirements(eventId):
    stub = f"result/retirements?eventId={eventId}"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_retirements = tablify(json_data)
    return df_retirements


getRetirements(eventId).head(3)

Fetching: https://api.wrc.com/content/result/retirements?eventId=365


,id,carNo,driverId,driverCountry,driverCountryImage,driver,coDriverId,coDriverCountry,coDriverCountryImage,coDriver,...,team/car,teamName,teamLogo,eligibility,groupClass,rallyId,entryId,controlId,reason,control
0,6768,#36,0a4d6b0a-f926-56f3-aca2-049f93add127,Japan,Flags/JPN.png,Tomoyuki SHINKAI,8e4b14a8-433b-5001-b586-0f5cf15edf1f,Japan,Flags/JPN.png,Yuichi ANDO,...,GR Yaris,Toyota,teamLogo/toyota.png,,JRCar1,369,47496,25795,ACCIDENT,SF1
1,6769,#36,0a4d6b0a-f926-56f3-aca2-049f93add127,Japan,Flags/JPN.png,Tomoyuki SHINKAI,8e4b14a8-433b-5001-b586-0f5cf15edf1f,Japan,Flags/JPN.png,Yuichi ANDO,...,GR Yaris,Toyota,teamLogo/toyota.png,,JRCar1,369,47496,25763,REJOINED,TC1B
2,6770,#6,d06b6c54-863a-533d-9241-9dd04f7acb85,Spain,Flags/ESP.png,Daniel SORDO,a1254acc-3431-5641-97c1-412d3713945c,Spain,Flags/ESP.png,Cándido CARRERA,...,i20 N Rally1 HYBRID,Hyundai,teamLogo/hyundai.png,M,RC1,369,47479,25798,ACCIDENT,SF2


In [246]:
def getChampionship(
    seasonId, eventId, championshipId, championship_type, championship="wrc"
):
    stub = f"result/championshipresult?seasonId={seasonId}&eventId={eventId}&championshipId={championshipId}&type={championship_type}&championship={championship}"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_splitTimes = tablify(json_data)
    return df_splitTimes


championship_type = "driver"  # where is list?
championshipId = None  # where is list?

getChampionship(seasonId, eventId, championshipId, championship_type).head(3)

Fetching: https://api.wrc.com/content/result/championshipresult?seasonId=20&eventId=365&championshipId=None&type=driver&championship=wrc


KeyError: 'values'

In [253]:
import sqlite_utils
#!rm wrc_2023_results.db
db = sqlite_utils.Database("wrc_2023_results.db")

In [248]:
# db["delme"].insert_all(df_fullcal.to_dict("records"))
# db["delme"].drop()

In [254]:
def db_add(table, df):
    if not df.empty:
        db[table].insert_all(df.to_dict("records"), alter=True)

db_add("fullcal", getFullCalendar())
db_add("resultscal", getResultsCalendar())

Fetching: https://api.wrc.com/content/filters/calendar?language=en&size=20&championship=wrc&origin=vcms&year=2023
Fetching: https://api.wrc.com/content/result/calendar?season=20&championship=wrc


In [255]:
# We can add alter=True to dynamically modify tables

def stageresults(row):
    stageId = row["stageId"]
    db_add("stagetimes",
        getStageTimes(eventId, rallyId, stageId))
    db_add("splittimes",
        getSplitTimes(eventId, rallyId, stageId))
    db_add("overall",
        getOverall(eventId, rallyId, stageId))


def rallyresults(row):
    eventId, rallyId = row[["eventId", "rallyId"]]
    db_add("retirements",getRetirements(eventId))
    db_add("penalties",getPenalties(eventId))
    db_add("startlist",getStartlist(eventId))
    db_add("itinerary",getItinerary(eventId))
    db_add("stagewinners",getStageWinners(eventId))

    df_stageDetails = getStageDetails(eventId, rallyId)
    db_add("stagedetails",df_stageDetails)
    df_stageDetails.apply(stageresults, axis=1)


_ = df_fullcal.apply(rallyresults, axis=1)

Fetching: https://api.wrc.com/content/result/retirements?eventId=353
Fetching: https://api.wrc.com/content/result/penalty?eventId=353
Fetching: https://api.wrc.com/content/result/startLists?eventId=353
namecols 5
Fetching: https://api.wrc.com/content/result/itinerary?eventId=353&extended=false
namecols 4
Fetching: https://api.wrc.com/content/result/stageWinners?eventId=353
Fetching: https://api.wrc.com/content/result/stages?eventId=353&rallyId=357&championship=wrc
Fetching: https://api.wrc.com/content/result/stageTimes?eventId=365&rallyId=369&stageId=SHD&championship=wrc
Fetching: https://api.wrc.com/content/result/splitTime?eventId=365&rallyId=369&stageId=SHD&championship=wrc
Fetching: https://api.wrc.com/content/result/stageResult?eventId=365&rallyId=369&stageId=SHD&championship=wrc
Fetching: https://api.wrc.com/content/result/stageTimes?eventId=365&rallyId=369&stageId=4161&championship=wrc
Fetching: https://api.wrc.com/content/result/splitTime?eventId=365&rallyId=369&stageId=4161&ch

In [207]:
getStageTimes(eventId, rallyId, stageId)

Fetching: https://api.wrc.com/content/result/stageTimes?eventId=365&rallyId=369&stageId=FINAL&championship=wrc


,id,pos,carNo,driverId,driverCountry,driverCountryImage,driver,coDriverId,coDriverCountry,coDriverCountryImage,...,team/car,teamName,teamLogo,eligibility,stageTime,penaltyTime,totalTime,diffFirst,diffPrev,groupClass
0,c991bcbd-6d9a-5efc-8bde-53d1c3bacae5,1,#33,ae7329c9-79b3-5d96-886c-22cca6217764,United Kingdom of Great Britain and Northern I...,Flags/GBR.png,Elfyn EVANS,53e56691-fe7c-5271-9dc5-8960df28b221,United Kingdom of Great Britain and Northern I...,Flags/GBR.png,...,GR Yaris Rally1 HYBRID,Toyota,teamLogo/toyota.png,M,3:32:08.8,,3:32:08.8,,,RC1
1,2141d91f-8620-5848-8873-32f48a37f2d7,2,#17,1cf9ade6-25b5-5586-8393-eacfeb943eae,France,Flags/FRA.png,Sébastien OGIER,867cd58f-3fe3-5290-9cc5-b21cb41c523c,France,Flags/FRA.png,...,GR Yaris Rally1 HYBRID,Toyota,teamLogo/toyota.png,M,3:32:26.5,1:00.0,3:33:26.5,+1:17.7,+1:17.7,RC1
2,0b585f28-25f4-5d7f-be34-6ef26f0e086e,3,#69,d8e4bbea-3af2-5486-9ad5-a445aaec573e,Finland,Flags/FIN.png,Kalle ROVANPERÄ,e4dd8a3f-00e9-59f7-9871-9337af6085d7,Finland,Flags/FIN.png,...,GR Yaris Rally1 HYBRID,Toyota,teamLogo/toyota.png,M,3:33:55.3,,3:33:55.3,+1:46.5,+28.8,RC1
3,0d9160d4-c333-5f36-93ae-42414f169e7e,4,#4,916f5b10-fee9-5b4c-b17b-6fbbc343cc3c,Finland,Flags/FIN.png,Esapekka LAPPI,2ef28d31-4a74-5794-b720-12c327c941b6,Finland,Flags/FIN.png,...,i20 N Rally1 HYBRID,Hyundai,teamLogo/hyundai.png,M,3:34:59.1,,3:34:59.1,+2:50.3,+1:03.8,RC1
4,34ed535a-5a7e-54c3-8c11-d4e174cc82f2,5,#18,298f93b1-b0ef-5af4-9f0c-e468d29abfd2,Japan,Flags/JPN.png,Takamoto KATSUTA,6347a621-eade-5fff-a7b4-7c4b73651e8a,Ireland,Flags/IRL.png,...,GR Yaris Rally1 HYBRID,Toyota,teamLogo/toyota.png,,3:34:09.1,1:10.0,3:35:19.1,+3:10.3,+20.0,RC1
5,66bc405c-6bee-5d7d-8f3e-eac0c7c656ac,6,#8,6632e7ca-34bf-55b8-9cad-d060000fa794,Estonia,Flags/EST.png,Ott TÄNAK,00a8a5c3-f7ba-5086-86df-1a59b7da7e26,Estonia,Flags/EST.png,...,Puma Rally1 HYBRID,Ford,teamLogo/ford.png,M,3:35:37.1,,3:35:37.1,+3:28.3,+18.0,RC1
6,7dff01c5-4889-54ca-ba24-403c167a980e,7,#20,093f0b06-7a05-5353-92ac-f5a68ae51f41,Norway,Flags/NOR.png,Andreas MIKKELSEN,5d0fb669-fb61-5d51-805b-d09c1ac7e36a,Norway,Flags/NOR.png,...,Fabia RS,Skoda,teamLogo/skoda.png,WRC2 (T/D),3:39:42.5,,3:39:42.5,+7:33.7,+4:05.4,RC2
7,cff4e5f5-e0af-57e1-b18e-500798ba37ea,8,#21,199a29d0-97f5-530f-9a5d-7ab45b1da0cc,ANA,Flags/ANA.png,Nikolay GRYAZIN,7d63ec40-0bb6-5da3-954e-846a2d40a9a1,ANA,Flags/ANA.png,...,Fabia RS,Skoda,teamLogo/skoda.png,WRC2 (T/DC/CC),3:40:58.4,,3:40:58.4,+8:49.6,+1:15.9,RC2
8,05ad4792-ac68-5da3-b92d-9ba8e08f02d6,9,#22,cede2cae-3446-54d8-b9d7-7e65651ebfd1,Poland,Flags/POL.png,Kajetan KAJETANOWICZ,a488aa5c-affc-5f48-a95e-0f400495fd71,Poland,Flags/POL.png,...,Fabia RS,Skoda,teamLogo/skoda.png,WRC2 (DC/CC),3:51:34.7,,3:51:34.7,+19:25.9,+10:36.3,RC2
9,9c2ec8ec-8a1a-5ecc-af2e-d9022011e097,10,#35,e00988f4-2f7f-55f1-8ec6-a0e790578911,Japan,Flags/JPN.png,Hiroki ARAI,4f879f5d-2cce-5127-bf65-eb4034aee5f6,Japan,Flags/JPN.png,...,208 Rally4,Peugeot,teamLogo/peugeot.png,,3:54:31.5,,3:54:31.5,+22:22.7,+2:56.8,RC4


In [258]:
for row in db.query("select DISTINCT(eventId) from stagedetails "):
    print(row)

{'eventId': '353'}
{'eventId': '354'}
{'eventId': '355'}
{'eventId': '356'}
{'eventId': '357'}
{'eventId': '358'}
{'eventId': '359'}
{'eventId': '360'}
{'eventId': '361'}
{'eventId': '362'}
{'eventId': '363'}
{'eventId': '364'}
{'eventId': '365'}
